# NB #0:  Sanity Check NB1 - 0x8b6-V1.0

The purpose of this notebook is to compare the accuracy of our model and its various ways of data gathering against actual pool history.

This simulation will compare three modes:

1- **Contract Call:** getting input data from user contract call and exact smart contract method( parsed from decoded LOG_CALL anonymous events that balancer uses to log all activity) and apply our model to get outputs

2- **Simplified:** getting input data from pool events (LOG_JOIN, LOG_EXIT, LOG_SWAP) parsed from BigQuery and applying our model to get outputs. This method should have more deviations because it's assuming the flavour of BPool join, swap, exit, joinswap and exitswap and using after the fact output data as input.

3- **Relay Output:** just adding and removing balances as seen in actual on-chain pool events

# A. System Context

### A1.1 System Specification
System specification details are available in the Balancer Simulations documentation:
- [Mathematical Specification](https://token-engineering-balancer.gitbook.io/balancer-simulations/additional-code-and-instructions/balancer-the-python-edition/balancer_math.py)
- [Model Architecture](https://token-engineering-balancer.gitbook.io/balancer-simulations/balancer-simulations/v10nboverview)
- [Naming Convention](https://token-engineering-balancer.gitbook.io/balancer-simulations/additional-code-and-instructions/naming-convention)

# B. cadCAD Simulations

### B1.1 Dependencies

In [ ]:
import pandas as pd 
from cadCAD.configuration.utils import config_sim

### B1.2 Initialize Pool


In this section you specify the Balancer Pool to be analyzed and run scripts to produce the'genesis_state', based on on-chain data. 

More information here: [Balancer Simulations documentation](https://token-engineering-balancer.gitbook.io/balancer-simulations/additional-code-and-instructions/onchaintransactions).  

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

from decimal import Decimal
parameters = {
    'spot_price_reference': ['DAI', 'DAI', 'DAI'],
    'decoding_type': ['REPLAY_OUTPUT', 'SIMPLIFIED', 'CONTRACT_CALL']
}
pp.pprint(parameters)

#### Import Pool Transactions and Initialize Pool state

Genesis state, pool transactions, and (external) USD token prices are pulled from on-chain/API data collected in a .json file.  
**Choose *your* Balancer Pool, and produce this .json file first, then add the file path below.**

Learn more how to create this file using parsing scripts in the [documentation](https://token-engineering-balancer.gitbook.io/balancer-simulations/v/master/additional-code-and-instructions/onchaintransactions).


In [ ]:
from model.genesis_states import generate_initial_state

initial_values = generate_initial_state(initial_values_json='data/0x8b6e6e7b5b3801fed2cafd4b22b8a16c2f2db21a-initial_pool_states-prices.json', spot_price_base_currency=parameters['spot_price_reference'][0])

#### State Variables and Initial Values

In [ ]:
print('## State Variables')
print('# Pool')
pool = initial_values['pool']
pp.pprint(initial_values)


#### External Token Prices

In [ ]:
print('# External token prices, initial state')
token_prices = initial_values['token_prices']
pp.pprint(token_prices)

print('# Action Type')
action_type = initial_values['action_type']
pp.pprint(action_type)


### B1.3 State Update Functions & Policies

Balancer Simulations replicate Balancer Pool Transactions in state update functions.  
For a detailed description, please visit the [Balancer Simulations documentation](https://token-engineering-balancer.gitbook.io/balancer-simulations/additional-code-and-instructions/balancer-the-python-edition).

### B1.4 Partial State Update Blocks

Partial State Update Blocks combine the following steps:  
1. **Initialize pool**, generate genesis state
2. Compute subsequent **actions in discrete timesteps**, store datetime and update the pool state variables
3. Update external **USD prices in discrete timesteps**, and store datetime  

For more information please visit the [Balancer Simulations documentation/Model Overview](https://token-engineering-balancer.gitbook.io/balancer-simulations/v/master/balancer-simulations/v10nboverview).  

To inject historical on-chain transactions to the model, reference the **actions.json of your pool below**.

In [ ]:
from model.partial_state_update_block import generate_partial_state_update_blocks

result = generate_partial_state_update_blocks('data/0x8b6e6e7b5b3801fed2cafd4b22b8a16c2f2db21a-actions-prices.json')
partial_state_update_blocks = result['partial_state_update_blocks']
pp.pprint(partial_state_update_blocks)

### B1.5 Configuration

[cadCAD simulations](https://github.com/cadCAD-org/cadCAD/blob/master/documentation/README.md) support Monte Carlo runs, and parameter sweeps which are not applied in this notebook and can be ignored here.  
Timesteps 'T' are defined by your pool's .json file, no need to make edits here.

In [ ]:
steps_number = result['steps_number']
print('# Steps ', steps_number)
sim_config = config_sim(
    {
        'N': 1,  # number of monte carlo runs
        'T': range(steps_number - 1),  # number of timesteps
        'M': parameters,  # simulation parameters
    }
)

### C1.5 Execution

In [ ]:

from model.sim_runner import *

df = run(initial_values, partial_state_update_blocks, sim_config)

### B1.7 Simulation Output Preparation

Post-processing (utils.py) splits up state variable dictionaries, and adds metrics to the data frame, such as 
- `token_total_value` (Total Value Locked in the pool)
- `invariant` (Invariant V of the pool)
- `generated_fees_(tokensymbol)` (fee collected per transaction)
- `token_k_values`  (token value in USD)  
based on  
- `token_k_balances`
- `token_k_price`  (external price feed)

In [ ]:
from model.parts.utils import post_processing

p_df = post_processing(df, include_spot_prices=False)

p_df

# D. Simulation Outcome – Verification of Results

Below we show a range of plots showing potential deviation of the simulation output vs. the actual on-chain pool balances (Relay Output).

Modes for reference:

1- **Contract Call:** getting input data from user contract call and exact smart contract method( parsed from decoded LOG_CALL anonymous events that balancer uses to log all activity) and apply our model to get outputs

2- **Simplified:** getting input data from pool events (LOG_JOIN, LOG_EXIT, LOG_SWAP) parsed from BigQuery and applying our model to get outputs. This method should have more deviations because it's assuming the flavour of BPool join, swap, exit, joinswap and exitswap and using after the fact output data as input.

3- **Relay Output:** just adding and removing balances as seen in actual on-chain pool events

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### D1.1 Token Balances

In [ ]:
k = 2 #define number of tokens in your pool
fig = make_subplots(rows=k, cols=1)

fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.token_dai_balance, line=dict(color='#4675ed'), name='token_dai_balance'), row=1, col=1)
fig.add_trace(go.Scatter(x=p_df.timestep, y=p_df.token_weth_balance, line=dict(color='#4145ab'), name='token_weth_balance'), row=2, col=1)
fig.update_layout(height=400, width=1000, title_text="<b>Token Balances in #</b>")
fig.show()

In [ ]:
# Simplified Mode = subset 0
# Contract Call Mode = subset 1
# Relay Output = subset 2

fig = px.scatter(p_df,x=p_df['timestep'], y=p_df['token_weth_balance'], facet_row="subset")
fig.update_layout(height=800, width=1000, title_text="<b>WETH Balances all modes</b>")
fig.show()

In [ ]:
# Simplified Mode = subset 0
# Contract Call Mode = subset 1
# Relay Output = subset 2


fig2 = px.scatter(p_df,x=p_df['timestep'], y=p_df['token_dai_balance'], facet_row="subset")
fig2.update_layout(height=800, width=1000, title_text="<b>DAI Balances all modes</b>")
fig2.show()

### D1.2 Deviations

In [ ]:
df_simplified = p_df[p_df['subset'] == 0]
df_simplified = df_simplified[['change_datetime','token_dai_balance', 'token_weth_balance']].copy()
df_simplified = df_simplified.rename(columns={"token_dai_balance" : "token_dai_balance_simplified", 'token_weth_balance': 'token_weth_balance_simplified' })
df_simplified = df_simplified.set_index('change_datetime')

df_contract_call = p_df[p_df['subset'] == 1]
df_contract_call = df_contract_call[['change_datetime','token_dai_balance', 'token_weth_balance']].copy()
df_contract_call = df_contract_call.rename(columns={"token_dai_balance" : "token_dai_balance_contract_call", 'token_weth_balance': 'token_weth_balance_contract_call' })
df_contract_call = df_contract_call.set_index('change_datetime')

df_reference = p_df[p_df['subset'] == 2] 
df_reference = df_reference[['change_datetime', 'token_dai_balance', 'token_weth_balance']].copy()
df_reference = df_reference.rename(columns={"token_dai_balance" : "token_dai_balance_reference", 'token_weth_balance': 'token_weth_balance_reference' })
df_reference =df_reference.set_index('change_datetime')

df_result = df_reference.join(df_contract_call).join(df_simplified)

df_result['token_dai_error_simplified'] = (df_result['token_dai_balance_simplified']-df_result['token_dai_balance_reference'])/df_result['token_dai_balance_reference']
df_result['token_weth_error_simplified'] = (df_result['token_weth_balance_simplified']-df_result['token_weth_balance_reference'])/df_result['token_weth_balance_reference']
df_result['token_dai_error_contract_call'] = (df_result['token_dai_balance_contract_call']-df_result['token_dai_balance_reference'])/df_result['token_dai_balance_reference']
df_result['token_weth_error_contract_call'] = (df_result['token_weth_balance_contract_call']-df_result['token_weth_balance_reference'])/df_result['token_dai_balance_reference']


df_result = df_result.reset_index()

df_result


In [ ]:
# DAI Deviation Simplified Mode vs. Relay Output

fig_diff_simplified_dai = px.scatter(p_df,x=df_result.index, y=df_result['token_dai_error_simplified'] )
fig_diff_simplified_dai.update_layout(height=800, width=1000, title_text="<b>DAI Deviation Simplified Mode vs. Relay Output</b>")
fig_diff_simplified_dai.show()

In [ ]:
# DAI Deviation Contract_Call Mode vs. Relay Output

fig_diff_contractcall_dai = px.scatter(p_df,x=df_result.index, y=df_result['token_dai_error_contract_call'])
fig_diff_contractcall_dai.update_layout(height=800, width=1000, title_text="<b>DAI Deviation Contract Call Mode vs. Relay Output</b>")
fig_diff_contractcall_dai.show()

In [ ]:
# WETH Deviation Simplified Mode vs. Relay Output


fig_diff_simplified_weth = px.scatter(p_df,x=df_result.index, y=df_result['token_weth_error_simplified'])
fig_diff_simplified_weth.update_layout(height=800, width=1000, title_text="<b>WETH Deviation Simplified Mode vs. Relay Output</b>")
fig_diff_simplified_weth.show()


In [ ]:
# WETH Deviation Contract_Call Mode vs. Relay Output


fig_diff_contractcall_weth = px.scatter(p_df,x=df_result.index, y=df_result['token_weth_error_contract_call'])
fig_diff_contractcall_weth.update_layout(height=800, width=1000, title_text="<b>WETH Deviation Contract Call Mode vs. Relay Output</b>")
fig_diff_contractcall_weth.show()
